<a href="https://colab.research.google.com/github/Deanazor/thefoodexplorer-ml/blob/master/Classification%20Model/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
training_dir = '/content/drive/MyDrive/Capstone_Project/Images/training/'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(training_dir, batch_size=8, target_size=(150,150))

testing_dir = '/content/drive/MyDrive/Capstone_Project/Images/testing/'
test_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_generator = test_datagen.flow_from_directory(testing_dir, batch_size=8, target_size=(150,150))

Found 2639 images belonging to 8 classes.
Found 377 images belonging to 8 classes.


In [ ]:
# model = Sequential()
# model.add(Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)))
# model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(32, (3,3), activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(64, (3,3), activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(7, activation='softmax'))

# model.summary()

In [ ]:
pre_trained = InceptionV3(include_top=False, input_shape=(150,150,3))

for layer in pre_trained.layers:
  layer.trainable = False

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
x = Flatten()(pre_trained.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(8, activation='softmax')(x)

model = Model(pre_trained.input, x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model.load_weights('/content/drive/MyDrive/Capstone_Project/model_8.h5')

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.1, 100, 0.1)
opt = tf.keras.optimizers.Adam()

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=300, verbose=1, validation_data=test_generator)

Epoch 1/300
330/330 [==============================] - 42s 114ms/step - loss: 0.8160 - accuracy: 0.7127 - val_loss: 1.0424 - val_accuracy: 0.7215
Epoch 2/300
330/330 [==============================] - 36s 108ms/step - loss: 0.7704 - accuracy: 0.7396 - val_loss: 0.8723 - val_accuracy: 0.7215
Epoch 3/300
330/330 [==============================] - 35s 105ms/step - loss: 0.8131 - accuracy: 0.7060 - val_loss: 1.0871 - val_accuracy: 0.7109
Epoch 4/300
330/330 [==============================] - 34s 103ms/step - loss: 0.7700 - accuracy: 0.7169 - val_loss: 0.9010 - val_accuracy: 0.7321
Epoch 5/300
330/330 [==============================] - 34s 103ms/step - loss: 0.8184 - accuracy: 0.7208 - val_loss: 0.9404 - val_accuracy: 0.7215
Epoch 6/300
330/330 [==============================] - 35s 106ms/step - loss: 0.7456 - accuracy: 0.7469 - val_loss: 0.9440 - val_accuracy: 0.7082
Epoch 7/300
330/330 [==============================] - 34s 104ms/step - loss: 0.8476 - accuracy: 0.6883 - val_loss: 0.9155 -

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('/content/drive/MyDrive/Capstone_Project/model_8.h5')

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/Capstone_Project/model/"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

In [ ]:
tf.keras.models.save_model(
    model_7,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Capstone_Project/model/1/assets

Saved model:
total 6259
drwx------ 2 root root    4096 May 25 03:39 assets
-rw------- 1 root root 6400598 May 25 03:39 saved_model.pb
drwx------ 2 root root    4096 May 25 03:39 variables


In [ ]:
! saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 150, 150, 3)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/usr/

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  42652      0 --:--:-- --:--:-- --:--:-- 42652
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/

In [ ]:
! {SUDO_IF_NEEDED} apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 74 not upgraded.
Need to get 326 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.5.1 [326 MB]
Fetched 326 MB in 5s (65.4 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.5.1_all.deb ...
Unpacking tensorflow-model-server (2.5.1) ...
Setting up tensorflow-model-server (2.5.1) ...


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
! nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=food_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 4 in a separate thread.


In [ ]:
! tail server.log

2021-05-25 03:41:48.500617: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /content/drive/MyDrive/Capstone_Project/model/1
2021-05-25 03:41:48.726324: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 1442114 microseconds.
2021-05-25 03:41:48.764794: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /content/drive/MyDrive/Capstone_Project/model/1/assets.extra/tf_serving_warmup_requests
2021-05-25 03:41:48.770177: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: food_model version: 1}
2021-05-25 03:41:48.771701: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-05-25 03:41:48.771840: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-05-25 03:41:48.772399: I tensorflow

In [ ]:
def predict(img):
  """
  Function to predict input image, image must be in 150 x 150 resolution
  Output will be the prediction string out of class indicies 
  """
  class_indicies = ['bakso', 'bika ambon', 'martabak manis', 'nasi goreng', 'rendang', 'sate', 'soto ayam']

  # image must be 150 x 150
  x=np.expand_dims(img, axis=0)
  images = np.vstack([x])
  images = images/255.0
  
  data = json.dumps({"signature_name": "serving_default", "instances": images.tolist()})
  # print(data)
  headers = {"content-type": "application/json"}
  json_response = requests.post('http://localhost:8501/v1/models/food_model:predict', data=data, headers=headers)

  # print(json_response.text)

  predictions = json.loads(json_response.text)['predictions']
  return class_indicies[np.argmax(predictions[0])]

In [ ]:
from google.colab import files
from keras.preprocessing import image
import numpy as np
import json
import requests

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  result = predict(img)
  print(result)
  # x=image.img_to_array(img)
  # x=np.expand_dims(x, axis=0)
  # images = np.vstack([x])
  # images = images/255.0
  
  # data = json.dumps({"signature_name": "serving_default", "instances": images.tolist()})
  # print(data)
  # headers = {"content-type": "application/json"}
  # json_response = requests.post('http://localhost:8501/v1/models/food_model:predict', data=data, headers=headers)

  # print(json_response.text)

  # predictions = json.loads(json_response.text)['predictions']
  # print(class_indicies[np.argmax(predictions[0])])
  # # print(classes[0])
  # # print(class_indicies[np.argmax(classes[0])])

Saving soto ayam_8.png to soto ayam_8.png
soto ayam
